En este ejercicio, el alumno tendrá que plantear y resolver un problema utilizando aprendizaje
automático. Se tendrán que llevar a cabo todos los pasos necesarios para la resolución del
problema: planteamiento, diseño, adquisición de los datos, análisis, entrenamiento de
algoritmos, evaluación, discusión de resultados, etc. Todo este proceso se desarrollará en un
cuaderno de Jupyter que conformará la entrega final.
El problema planteado podrá ser inventado o real, y puede estar basado en un problema de
negocio, un artículo científico, una competición, etc. Existen únicamente dos requisitos en
cuanto al caso de uso:
1. Tiene que utilizarse un conjunto de datos para su resolución.
2. La solución debe estar basada en aprendizaje automático, es decir, se debe haber
entrenado al menos un algoritmo.

![nba-logo](images/nba-logo.png)

In [65]:
import pandas as pd 
import numpy as np
from IPython.display import display
import zipfile as zp
import matplotlib.pyplot as plt
import sklearn.preprocessing as prep
from sklearn.metrics import precision_score, recall_score, confusion_matrix, mean_absolute_error, mean_absolute_percentage_error, r2_score, accuracy_score, mean_squared_error

Carga de los dataframes

In [66]:
zf = zp.ZipFile('dataframes/nba.zip')
df_games = pd.read_csv(zf.open('games.csv'))
df_games_details = pd.read_csv(zf.open('games_details.csv', low_memory=False))
df_players = pd.read_csv(zf.open('players.csv'))
df_ranking = pd.read_csv(zf.open('ranking.csv'))
df_teams = pd.read_csv(zf.open('dataframes/teams.csv'))

KeyError: "There is no item named 'dataframes/games.csv' in the archive"

Muestra varias filas de cada dataframe para realizar un primer analisis

In [ ]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.head())
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.head())
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.head())
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.head())
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.head())

Muestra filas y columnas de cada dataframe

In [ ]:
print("--------- PARTIDOS ----------")
print("-----------------------------")
display(df_games.shape)
print("--------- DETALLES PARTIDOS ----------")
print("--------------------------------------")
display(df_games_details.shape)
print("--------- JUGADORES ----------")
print("------------------------------")
display(df_players.shape)
print("--------- RANKING LIGA ----------")
print("---------------------------------")
display(df_ranking.shape)
print("--------- EQUIPOS ----------")
print("----------------------------")
display(df_teams.shape)